In [16]:
import numpy as n
import openpyxl
from scipy.stats import binom
import math

In [17]:
## Import PoF and Failure Time
def Update():
  # Model_List must be imported every time
  data = openpyxl.load_workbook("Model_List.xlsx")
  read_data = data.active
  Probability = []
  Fail_Time = []
  for row in range(1, read_data.max_row):
      for col in read_data.iter_cols(2,2):
        Probability.append(col[row].value)
  for row in range(1, read_data.max_row):
      for col in read_data.iter_cols(3,3):
        Fail_Time.append(col[row].value)
  Probability = [0.99931506849315, 0.999868149, 0.999868149, 0.999868149, 0.999868149, 1, 1, 1, 1, 0.999996546, 0.999996546, 1, 1, 1, 1, 0.999994046, 0.999994046, 0.9999, 0.999996546, 1, 1, 0.999982209, 0.9999, 0.9999, 0.9999, 0.999982209, 0.999982209, 0.9999, 0.999982209, 0.999982209, 0.999247479 , 0.999897429, 0.999247479 , 0.999897429, 0.999247479 , 0.999897429, 1, 0.999312485, 0.999312485, 0.999312485, 0.999312485, 0.999312485, 0.999312485, 0.999897429, 0.999897429, 0.999989539, 0.999897429, 0.999897429, 0.999875595, 0.999875595]
  return [Probability, Fail_Time]


In [25]:
## import weibull distro
from scipy.stats import weibull_min

## tests using Weibull distro
# def test(Probability):
#   return weibull_min.rvs(Probability,1)

## Tests each probability given a binomial distribution
def test(Probability):
  return binom.rvs(1,Probability)


In [26]:
## Tests the Electrical side of the model. This is checked before HVAC.
def Failure_Elec(PoF,Carry):
  # 1,2,3,4 are the generators, only 2 are needed
  if(PoF == 1 or PoF == 2 or PoF == 3 or PoF == 4):
    # Checks if there are 3 gens that fail
    if (Carry[1] + Carry[2] + Carry[3] + Carry[4]) == 3:
      Carry[PoF] += 1
      return [0,Carry]
    # If less than 2 fail, increases the carry
    Carry[PoF] += 1
    return [1,Carry]
  # Checks the busses, only one needed, same process as gens
  elif(PoF == 5 or PoF == 6):
    if (Carry[5] or Carry[6]) == 1:
      Carry[PoF] += 1
      return [0,Carry]
    Carry[PoF] += 1
    return [1,Carry]
  # Checks UPS, only one needed
  elif(PoF == 7 or PoF == 8):
    if (Carry[7] or Carry[8]) == 1:
      Carry[PoF] += 1
      return [0,Carry]
    Carry[PoF] += 1
    return [1,Carry]
  # If none of the components are those listed above, there is no exception
  # therefor they fail
  else:
    return [0,Carry]

In [27]:
# Checks HVAC side, same as electrical]
def Failure_HVAC(PoF,Carry):
  # Check for Watertowers
  if(PoF == 30 or PoF == 32 or PoF == 34):
    Carry[PoF] += 1
    return [1,Carry]
  # Checks Pumps
  elif(PoF == 31 or PoF == 33 or PoF == 35):
    Carry[PoF] += 1
    # Checks if those pumps that are connected to water towers are down together
    if((Carry[30]==0 and Carry[31]==0) or (Carry[32]==0 and Carry[33]==0)
      or (Carry[34]==0 and Carry[35]==0)):
      return [1,Carry]
    return [0,Carry]
  # Checks Direct Chillers
  elif(PoF == 40 or PoF == 37 or PoF == 38 or PoF == 39):
    if (Carry[40] + Carry[37] + Carry[38] + Carry[39]) == 4:
      Carry[PoF] += 1
      return [0,Carry]
    Carry[PoF] += 1
    return [1,Carry]
  # Checks Backup Chillers
  elif(PoF == 41 or PoF == 42):
    if (Carry[41] or Carry[42]) == 1:
      Carry[PoF] += 1
      return [0,Carry]
    Carry[PoF] += 1
    return [1,Carry]
  # Checks CRAH's
  elif(PoF =="48" or PoF=="49"):
    if (Carry[48] or Carry[49]) == 1:
      Carry[PoF] += 1
      return [0,Carry]
    Carry[PoF] += 1
    return [1,Carry]
  else:
    return [0,Carry]

In [28]:
## Runs the Simulation
def Simulation():
  # Allows for the import of the list of PoF and Failtime
  pull = Update()
  Prob = pull[0]
  Fail_Time = pull[1]
  # Sets the hour
  hour = 0
  # Sets the list for each hour
  Year_Output = []
  # Creates an empty list with the amount of components to count failures
  Fail_List = [0]*len(Prob)
  while hour < (24*365):
    i = 0
    test_mult = 1
    elec_mult = 0
    Comp_Test=[]
    # Tests every component and adds them to the Comp_Test list
    while i < len(Prob):
      temp = test(Prob[i])
      Comp_Test.append(temp)
      test_mult *= temp
      # Does the comp for only electrical
      if i == 29:
        elec_mult = test_mult
      i+=1
    # If every component passes, passes the hour
    if test_mult == 1 :
      Year_Output.extend([1])
      hour+=1
    else:
      # Sees if positions need to be skipped
      if (Comp_Test[0] == 1):
        # If all electrical is good, start with HVAC
        if elec_mult == 1:
          i = 29
        # Else, start after gens since com power is good
        else:
          i = 5
      # If Com Power is down, start with Com Power
      else:
        i = 0
      Fail_Carry = [0]*len(Prob)
      # Tests Electrical slide, last lectrical comp is 29
      while i < 30:
        # Does comp Fail
        if Comp_Test[i] == 0:
          temp = Fail_List[i]
          Fail_List[i] = temp + 1
          con = Failure_Elec(i,Fail_Carry)
          # Gets info from Failure_Elec and sees if failed
          if (con[0] == 0 and i!=0):
            Year_Output.extend([0]*Fail_Time[i])
            hour += Fail_Time[i]
            i = 30
          # Keeps the carry and continues
          else:
            Fail_Carry = con[1]
        i += 1
        # Catches loop if all comps work, starting with 30
        if i == 30:
          while i < (len(Prob)):
            if Comp_Test[i] == 0:
              temp = Fail_List[i]
              Fail_List[i] = temp + 1
              con = Failure_HVAC(i,Fail_Carry)
              if (con[0] == 0):
                Year_Output.extend([0]*Fail_Time[i])
                hour += Fail_Time[i]
                i = 49
              else:
                Fail_Carry = con[1]
            i+=1
  # Returns the Total amount of hours passed in a year and the list of fails
  return [sum(Year_Output) , Fail_List]


In [29]:
from numpy.polynomial.polynomial import polyval3d
def run_multi(years):
  i = 0
  Output = 0
  List = []
  while i < years:
    temp = Simulation()
    Output = Output + temp[0]
    List.append(temp[1])
    i+=1
  Fail_List = [0]*50
  for row in range(years):
    for col in range(len(Fail_List)):
      temp = Fail_List[col]
      Fail_List[col] = temp + List[row][col]/years
  Output = (Output*100)/(years*24*365)
  print("The sytem is up " + str(Output) + "% of the time")
  return Fail_List
 
  
  
 
 

In [30]:
def create_t(List):
 Final_Fail = []
 data = openpyxl.load_workbook("Model_List.xlsx")
 read_data = data.active
 i = -1
 for row in range(1, read_data.max_row):
     i +=1
     for col in read_data.iter_cols(1,1):
        temp_name = col[row].value
        Final_Fail.append([List[i],temp_name])
        if(i == 49):
            return Final_Fail

In [31]:
Fail_List = run_multi(1)
Fail_Tup = create_t(Fail_List)

# Number of comps to be displayed
i = 3
print(" Fail")
print("Amount     Name")
while i > 0:
    temp = max(Fail_Tup)
    print(temp)
    Fail_Tup.remove(temp)
    i-=1


The sytem is up 99.45205479452055% of the time
 Fail
Amount     Name
[8.0, 'Chiller 3']
[7.0, 'Chiller 5']
[6.0, 'Chiller 2']
